In [16]:
from utils.PandasToolsFunction import *
from utils.DataPreprocessingTools import *
from utils.loss_functions import mean_absolute_percentage_error, mean_absolute_percentage_error_np

from torch.nn.functional import mse_loss

import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import numpy as np
import torch 
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

from torch.utils.data import Dataset, DataLoader

from tqdm.notebook import tqdm

import json

import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import BaggingRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [17]:
train_data = pd.read_csv('data/train_data_day_typed_cov_replaced_all_features.csv')
train_data['date'] = pd.to_datetime(train_data['date'])
train_data = date_filter(train_data, start_date='2019-12-31', end_date='2022-12-31')

In [18]:
class LR_model():
    def __init__(self, train_data, test_size):
        self.train_data = train_data
        self.stations = train_data['station'].unique()
        self.datasets = {}
        self.lrs = {}
        self.features_col = ['week', 'day_numeric', 'day_type']

        for station in self.stations :
            # create datasets
            station_indices = train_data.index[train_data['station'] == station]
            features = train_data.loc[station_indices, self.features_col]
            features = features.values
            y = train_data.loc[station_indices, 'y']
            y = np.array(y)
            #X_normalized = self.scaler.transform(features)
            self.datasets[station] = train_test_split(features, y, test_size=test_size, random_state= 42, shuffle=True)

            # creates LR models
            #self.lrs[station] = LinearRegression() # err: 742.88
            #self.lrs[station] = BaggingRegressor(LinearRegression(), n_estimators=10, random_state=42) # err: 737.17
            #self.lrs[station] = AdaBoostRegressor(LinearRegression(), n_estimators=300, random_state=42) # err: 756.77
            self.lrs[station] = DecisionTreeRegressor() # err: 71.98
            
            #self.lrs[station] = RandomForestRegressor(n_estimators=10, max_depth=20) # err: 78.01
            #self.lrs[station] = AdaBoostRegressor(DecisionTreeRegressor(max_depth=25), n_estimators=20, learning_rate=0.1, loss='linear', random_state=42) # err: 106.58

            #self.lrs[station] = GradientBoostingRegressor(random_state=42) # err: 316.45

            # self.lrs[station] = MLPRegressor(hidden_layer_sizes=[3, 3], random_state=42) # err: 436.56
            # self.lrs[station] = MLPRegressor(hidden_layer_sizes=[10], random_state=42) # err: 499.51
            #self.lrs[station] = MLPRegressor(hidden_layer_sizes=[3, 3], learning_rate = 'adaptive', learning_rate_init = 0.001, max_iter = 1000, random_state=42) # err: 427

            """
            estimators = [('ridge', RidgeCV()),
                          ('lasso', LassoCV(random_state=42)),
                          ('knr', KNeighborsRegressor(n_neighbors=20,metric='euclidean'))]
            final_estimator = GradientBoostingRegressor(n_estimators=25, subsample=0.5, min_samples_leaf=25, max_features=1, random_state=42)
            self.lrs[station] = StackingRegressor(estimators=estimators, final_estimator=final_estimator)
            """
            """
            estimators = [('lr', LinearRegression()),
                          ('dt', DecisionTreeRegressor(max_depth=20)),
                          ('mlp', MLPRegressor(hidden_layer_sizes=[3,3], random_state=42))]
            final_estimator = MLPRegressor(hidden_layer_sizes=[3,3], random_state=42)
            self.lrs[station] = StackingRegressor(estimators=estimators, final_estimator=final_estimator)
            """

    def train(self, normalize=False, polynomial=False):
        err = 0
        for station in tqdm(self.stations):
            X_train, X_test, y_train, y_test = self.datasets[station]

            if polynomial:
                poly_features = PolynomialFeatures(degree=2)
                X_train = poly_features.fit_transform(X_train)
                X_test = poly_features.transform(X_test)

            if normalize:
                X_train = X_train.astype(np.float64)
                X_train[:,0] /= 52
                X_train[:,1] /= 7
                X_test = X_test.astype(np.float64)
                X_test[:,0] /= 52
                X_test[:,1] /= 7
            
            self.lrs[station].fit(X_train, y_train)

            test_pred = self.lrs[station].predict(X_test)
            err += mean_absolute_percentage_error_np(y_test, test_pred).item()
        return err

In [19]:
test_size = 0.01
lr_model = LR_model(train_data, test_size)

In [20]:
err = lr_model.train()
print("err:", err)

  0%|          | 0/439 [00:00<?, ?it/s]

err: 71.98803072059476


In [21]:
test_data = pd.read_csv('data/test_data_day_typed_cov_replaced_all_features.csv')
test_data['date'] = pd.to_datetime(test_data['date'])
test_data

,index,date,week,day_name,day_numeric,station,day_type
0,2023-01-01_1J7,2023-01-01,52,Sunday,6,1J7,7
1,2023-01-01_O2O,2023-01-01,52,Sunday,6,O2O,7
2,2023-01-01_8QR,2023-01-01,52,Sunday,6,8QR,7
3,2023-01-01_L58,2023-01-01,52,Sunday,6,L58,7
4,2023-01-01_UMC,2023-01-01,52,Sunday,6,UMC,7
...,...,...,...,...,...,...,...
78647,2023-05-31_N9K,2023-05-31,22,Wednesday,2,N9K,0
78648,2023-05-31_P6E,2023-05-31,22,Wednesday,2,P6E,0
78649,2023-05-31_BDC,2023-05-31,22,Wednesday,2,BDC,0
78650,2023-05-31_QD6,2023-05-31,22,Wednesday,2,QD6,0


In [22]:
for index, row in tqdm(test_data.iterrows()):
    features = row[lr_model.features_col]
    features = features.values
    features = [features]
    y_pred = lr_model.lrs[row['station']].predict(features)
    print("y_pred:", y_pred)

0it [00:00, ?it/s]

y_pred: [37.]
y_pred: [24.]
y_pred: [23.]
y_pred: [5.]
y_pred: [39.]
y_pred: [122.]
y_pred: [24.]
y_pred: [917.]
y_pred: [65.]
y_pred: [146.]
y_pred: [43.]
y_pred: [84.]
y_pred: [73.]
y_pred: [257.]
y_pred: [210.]
y_pred: [38.]
y_pred: [160.]
y_pred: [111.]
y_pred: [170.]
y_pred: [15.]
y_pred: [101.]
y_pred: [78.]
y_pred: [63.]
y_pred: [399.]
y_pred: [68.]
y_pred: [62.]
y_pred: [38.]
y_pred: [40.]
y_pred: [1459.]
y_pred: [1440.]
y_pred: [555.]
y_pred: [3570.]
y_pred: [61.]
y_pred: [79.]
y_pred: [577.]
y_pred: [405.]
y_pred: [1079.]
y_pred: [867.]
y_pred: [2905.]
y_pred: [252.]
y_pred: [781.]
y_pred: [701.]
y_pred: [39.]
y_pred: [1435.]
y_pred: [282.]
y_pred: [27.]
y_pred: [236.]
y_pred: [57.]
y_pred: [94.]
y_pred: [175.]
y_pred: [37.]
y_pred: [1118.]
y_pred: [65.]
y_pred: [25.]
y_pred: [100.]
y_pred: [103.]
y_pred: [1163.]
y_pred: [32.]
y_pred: [19.]
y_pred: [412.]
y_pred: [10.]
y_pred: [7.]
y_pred: [36.]
y_pred: [5.]
y_pred: [119.]
y_pred: [5221.]
y_pred: [17.]
y_pred: [38.]
y_pred: [

KeyboardInterrupt: 